In [1]:
import sys
import sys
sys.path.append('../build/')

import numpy as np
np.set_printoptions(precision=6, suppress=True)

import versor as vsr

versor/cga.py:4: RuntimeWarning: to-Python converter for vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)1, (short)2, (short)4, (short)8, (short)16> > already registered; second conversion method ignored.
  from libversor import (Biv, Bst, Cir, Con, Dil, Dll, Dlp, Dls, Drb, Drt, Drv,


# PK8103 Swarm Intelligence

This notebook implements the swarm intelligence algorithms **Ant Colony Optimization (ACO)**, **Artificial Bee Colony Optimization (ABC)** and **Particle Swarm Optimization (PSO)** for estimation of **rigid body motion motors** from point clouds using **Conformal Geometric Algebra (CGA)**.

## Conformal Geometric Algebra

### Polar Decomposition

$$M = \frac{X}{|X|}\left(1 - \frac{\langle X \widetilde{X} \rangle_4}{2 \langle X \widetilde{X} \rangle} \right),\ X \in \mathbb{M}$$

In [2]:
def polar_decomposition(x):
    xx = x * x.rev()
    sinv = (vsr.Sca(1.0) + (vsr.Drt(- xx[7] / (2 * xx[0]))))
    return x * sinv

### Cost Function

$$ C = \sum_i^n \| \mathcal{K}^{-1}(b_i) - \mathcal{K}^{-1}(M a_i \widetilde{M})    \|^2 $$

In [3]:
def cost_function(m):
    mot = vsr.Mot(*m)
#     print(mot)
    mot = polar_decomposition(mot)
    b_ = [point.spin(mot) for point in points_a]
    b = points_b
    return np.sum([np.linalg.norm(pa.to_array()[:3] - 
                                  pb.to_array()[:3]) for pa, pb in zip(b, b_)])

### Dataset generation

In [4]:
n = 10
motor = vsr.Trs.from_vector(vsr.Vec(1,1,1)) * vsr.Rot.from_bivector(vsr.Biv(0,1,0) * np.pi/6.0)
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n)]
points_b = [point.spin(motor) for point in points_a]

## Artificial Bee Colony Optimization *(ABC)*

In [18]:
from PyGMO.problem import base

class MotorEstimationProblem(base):

    def __init__(self, dim=8):
        # First we call the constructor of the base class telling PyGMO
        # what kind of problem to expect ('dim' dimensions, 1 objective, 0 contraints etc.)
        super(MotorEstimationProblem,self).__init__(dim)

        # We set the problem bounds (in this case equal for all components)
        self.set_bounds(-1, 1)

    # Reimplement the virtual method that defines the objective function.
    def _objfun_impl(self, x):

        # Note that we return a tuple with one element only. In PyGMO the objective functions
        # return tuples so that multi-objective optimization is also possible.
        return (cost_function(x), )

    # Finally we also reimplement a virtual method that adds some output to the __repr__ method
    def human_readable_extra(self):
        return "\n\t Problem dimension: " + str(self.__dim)

In [37]:
from PyGMO import algorithm, island

prob = MotorEstimationProblem(dim=8)  # Create a 10-dimensional problem
algo = algorithm.bee_colony(gen=2000)  # 500 generations of bee_colony algorithm
isl = island(algo, prob, 100)  # Instantiate population with 20 individuals
isl.evolve(1)  # Evolve the island once
isl.join()
print(isl.population.champion.f[0])
x = isl.population.champion.x
m = polar_decomposition(vsr.Mot(*x))
print(m)

0.000617144157239
-0.965925  1.70338e-05  0.258817  -4.02722e-05  0.612374  0.482946  0.353568  0.129423  


## Particle Swarm Optimization *(PSO)*

In [28]:
prob = MotorEstimationProblem(dim=8)  # Create a 10-dimensional problem
algo = algorithm.pso(gen=500)  # 500 generations of bee_colony algorithm
isl = island(algo, prob, 20)  # Instantiate population with 20 individuals
isl.evolve(1)  # Evolve the island once
isl.join()
print(isl.population.champion.f[0])
x = isl.population.champion.x
m = polar_decomposition(vsr.Mot(*x))
print(m)

(5.28475486091371e-09,)
-0.965926  1.11723e-10  0.258819  -3.73612e-11  0.612372  0.482963  0.353553  0.12941  


In [ ]:
prob = my_problem(dim=8)  # Create a 10-dimensional problem
algo = algorithm.de(gen=500)  # 500 generations of bee_colony algorithm
isl = island(algo, prob, 200)  # Instantiate population with 20 individuals
isl.evolve(1)  # Evolve the island once
isl.join()
print(isl.population.champion.f[0])
x = isl.population.champion.x
m = polar_decomposition(vsr.Mot(*x))
print(m)

## Ant Colony Optimization for Continuous Domains *(ACO$_\mathbb{R}$)*